# Sensitivity Analysis (Policies Sampling)


In [1]:
from __future__ import (unicode_literals, print_function, absolute_import, division)
from ema_workbench import (Model, SequentialEvaluator, MultiprocessingEvaluator, Policy, Scenario)
from ema_workbench.em_framework.evaluators import perform_experiments
from ema_workbench.em_framework.samplers import sample_uncertainties
from ema_workbench.util import ema_logging
ema_logging.log_to_stderr(ema_logging.INFO)
import time
from prob_form import get_model_for_problem_formulation
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import scipy as sp
from ema_workbench.em_framework.evaluators import LHS, SOBOL, MORRIS
from ema_workbench.analysis import feature_scoring
from ema_workbench.analysis.scenario_discovery_util import RuleInductionType
from ema_workbench.em_framework.salib_samplers import get_SALib_problem
from SALib.analyze import sobol

In [2]:
dike_model, planning_steps = get_model_for_problem_formulation(7)

In [3]:
from ema_workbench import load_results

experiments_loaded, outcomes_loaded = load_results('sobol_uncertainties_sampling.tar.gz')

[MainProcess/INFO] results loaded succesfully from C:\Users\lekha\Desktop\ReferenceMDM\epa1361_open\final assignment\sobol_uncertainties_sampling.tar.gz


### Setting Reference Scenarios

From the range of uncertainties, we will create scenarios based on changing uncertainties that matter the most (Derived from sensitivity analysis, uncertainties sampling)

In [4]:
for unc in dike_model.uncertainties:
    print(repr(unc))

CategoricalParameter('discount rate 0', [0, 1, 2, 3])
CategoricalParameter('discount rate 1', [0, 1, 2, 3])
CategoricalParameter('discount rate 2', [0, 1, 2, 3])
IntegerParameter('A.0_ID flood wave shape', 0, 132)
RealParameter('A.1_Bmax', 30, 350)
RealParameter('A.1_pfail', 0, 1)
CategoricalParameter('A.1_Brate', [0, 1, 2])
RealParameter('A.2_Bmax', 30, 350)
RealParameter('A.2_pfail', 0, 1)
CategoricalParameter('A.2_Brate', [0, 1, 2])
RealParameter('A.3_Bmax', 30, 350)
RealParameter('A.3_pfail', 0, 1)
CategoricalParameter('A.3_Brate', [0, 1, 2])
RealParameter('A.4_Bmax', 30, 350)
RealParameter('A.4_pfail', 0, 1)
CategoricalParameter('A.4_Brate', [0, 1, 2])
RealParameter('A.5_Bmax', 30, 350)
RealParameter('A.5_pfail', 0, 1)
CategoricalParameter('A.5_Brate', [0, 1, 2])


In [5]:
#Defining reference scenarios
#Worst case: Flood-inducing scenario - Global warming + High probability of ALL Dikes failing along 
#with economic implications(through discount rates)

ref_high_dikefailure_prob = Scenario('ref_high_dikefailure_prob', **{   
    'A.0_ID flood wave shape':130,
    'A.1_Bmax':300,
    'A.1_Brate':10,
    'A.1_pfail':experiments_loaded['A.1_pfail'].quantile(0.9),
    'A.2_Bmax':300,
    'A.2_Brate':5,
    'A.2_pfail':experiments_loaded['A.2_pfail'].quantile(0.9),
    'A.3_Bmax':300,
    'A.3_Brate':10,  
    'A.3_pfail':experiments_loaded['A.3_pfail'].quantile(0.9),
    'A.4_Bmax':300,
    'A.4_Brate':10,
    'A.4_pfail':experiments_loaded['A.4_pfail'].quantile(0.9),
    'A.5_Bmax':300,
    'A.5_Brate':10,
    'A.5_pfail':experiments_loaded['A.5_pfail'].quantile(0.9),
    'discount rate 0':2,
    'discount rate 1':2,
    'discount rate 2':2
        })

In [6]:
#THINGS WE TRIED BUT DIDNT WORK 

# from ema_workbench import MultiprocessingEvaluator, SequentialEvaluator, ema_logging
# from ema_workbench.em_framework.evaluators import BaseEvaluator
# from ema_workbench.em_framework.optimization import (HyperVolume,
#                                                      EpsilonProgress)
# ema_logging.log_to_stderr(ema_logging.INFO)

# def optimize(scenario, nfe, model, converge_metrics, epsilons):
#     with SequentialEvaluator(model) as evaluator:
#         results, convergence = evaluator.optimize(nfe=nfe, searchover='levers',
#                                      convergence=convergence_metrics,
#                                      epsilons=epsilons,
#                                      reference=scenario)
#     return results, convergence

# results = []
# # convergence_metrics = [HyperVolume.from_outcomes(dike_model.outcomes),
# #                        EpsilonProgress()]

# convergence = [HyperVolume(minimum=[0]*11, maximum=[1.75e-05, 175094.55, 289010644.28, 1.75e-05, 175094.55, 367832359.71, 0.00095, 142575315.00, 5077927.25,1644699000.0,336.2204869362749]),
#                EpsilonProgress()]

# # constraints = [Constraint("expected deaths A1", outcome_names="A.1_Expected Number of Deaths 0",function=lambda x:max(0, 1.75e-05)),
# #                Constraint("expected damage A1", outcome_names="A.1_Expected Annual Damage 0",function=lambda x:max(0, 175094.55)),
# #                Constraint("dike investment A1", outcome_names="A.1_Dike Investment Costs 0",function=lambda x:max(0, 289010644.28)),
# #                Constraint("expected deaths A2", outcome_names="A.2_Expected Number of Deaths 0",function=lambda x:max(0, 1.75e-05)),
# #                Constraint("expected damage A2", outcome_names="A.2_Expected Annual Damage 0",function=lambda x:max(0, 175094.55)),
# #                Constraint("dike investment A2", outcome_names="A.2_Dike Investment Costs 0",function=lambda x:max(0, 367832359.71)),
# #                Constraint("expected deaths A3", outcome_names="A.3_Expected Number of Deaths 0",function=lambda x:max(0, 0.00095)),
# #                Constraint("expected damage A3", outcome_names="A.3_Expected Annual Damage 0",function=lambda x:max(0, 142575315.00)),
# #                Constraint("dike investment A3", outcome_names="A.3_Dike Investment Costs 0",function=lambda x:max(0, 5077927.25)),
# #                Constraint("rfr total costs", outcome_names="RfR Total Costs",function=lambda x:max(0, 1644699000.0)),
# #                Constraint("expected evacuation costs", outcome_names="Expected Evacuation Costs",function=lambda x:max(0, 336.2204869362749)),
# #                Constraint("expected deaths A1", outcome_names="A.1_Expected Number of Deaths 1",function=lambda x:max(0, 1.75e-05)),
# #                Constraint("expected damage A1", outcome_names="A.1_Expected Annual Damage 1",function=lambda x:max(0, 175094.55)),
# #                Constraint("dike investment A1", outcome_names="A.1_Dike Investment Costs 1",function=lambda x:max(0, 289010644.28)),
# #                Constraint("expected deaths A2", outcome_names="A.2_Expected Number of Deaths 1",function=lambda x:max(0, 1.75e-05)),
# #                Constraint("expected damage A2", outcome_names="A.2_Expected Annual Damage 1",function=lambda x:max(0, 175094.55)),
# #                Constraint("dike investment A2", outcome_names="A.2_Dike Investment Costs 1",function=lambda x:max(0, 367832359.71)),
# #                Constraint("expected deaths A3", outcome_names="A.3_Expected Number of Deaths 1",function=lambda x:max(0, 0.00095)),
# #                Constraint("expected damage A3", outcome_names="A.3_Expected Annual Damage 1",function=lambda x:max(0, 142575315.00)),
# #                Constraint("dike investment A3", outcome_names="A.3_Dike Investment Costs 1",function=lambda x:max(0, 5077927.25)),
# #                Constraint("rfr total costs", outcome_names="RfR Total Costs",function=lambda x:max(0, 1644699000.0)),
# #                Constraint("expected evacuation costs", outcome_names="Expected Evacuation Costs",function=lambda x:max(0, 336.2204869362749)),
# #                Constraint("expected deaths A1", outcome_names="A.1_Expected Number of Deaths 2",function=lambda x:max(0, 1.75e-05)),
# #                Constraint("expected damage A1", outcome_names="A.1_Expected Annual Damage 2",function=lambda x:max(0, 175094.55)),
# #                Constraint("dike investment A1", outcome_names="A.1_Dike Investment Costs 2",function=lambda x:max(0, 289010644.28)),
# #                Constraint("expected deaths A2", outcome_names="A.2_Expected Number of Deaths 2",function=lambda x:max(0, 1.75e-05)),
# #                Constraint("expected damage A2", outcome_names="A.2_Expected Annual Damage 2",function=lambda x:max(0, 175094.55)),
# #                Constraint("dike investment A2", outcome_names="A.2_Dike Investment Costs 2",function=lambda x:max(0, 367832359.71)),
# #                Constraint("expected deaths A3", outcome_names="A.3_Expected Number of Deaths 2",function=lambda x:max(0, 0.00095)),
# #                Constraint("expected damage A3", outcome_names="A.3_Expected Annual Damage 2",function=lambda x:max(0, 142575315.00)),
# #                Constraint("dike investment A3", outcome_names="A.3_Dike Investment Costs 2",function=lambda x:max(0, 5077927.25)),
# #                Constraint("rfr total costs", outcome_names="RfR Total Costs",function=lambda x:max(0, 1644699000.0)),
# #                Constraint("expected evacuation costs", outcome_names="Expected Evacuation Costs",function=lambda x:max(0, 336.2204869362749)),
# #               ]

# epsilons = [0.08,]*len(dike_model.outcomes)

# results.append(optimize(ref_high_dikefailure_prob, 10, dike_model, convergence_metrics, epsilons))

# from ema_workbench.em_framework.optimization import EpsilonProgress
# convergence = [EpsilonProgress()]
# epsilons = [0.1,]*len(dike_model.outcomes)

# with SequentialEvaluator(dike_model) as evaluator:
#     result, convergence = evaluator.optimize(nfe=5, searchover='levers',
#                                     epsilons=epsilons,
#                                     )

### Search over levers
Directed search is most often used to search over the decision levers in order to find good candidate strategies. This is for example the first step in the Many Objective Robust Decision Making process. This is straightforward to do with the workbench using the optimize method.

The result from optimize is a DataFrame with the decision variables and outcomes of interest.

In [7]:
#UNCOMMENT WHEN RUNNING  FOR THE FIRST TIME 

# with MultiprocessingEvaluator(dike_model) as evaluator:
#     results_ref1 = evaluator.optimize(nfe=5000, epsilons=[0.1]*12, searchover='levers', reference=ref_high_dikefailure_prob,
#                                )

In [8]:
# results_ref1

In [9]:
# results_ref1.to_csv(r'scenario_ref1_optimal_policies_new.csv',index = False)

In [10]:
results_load = pd.read_csv('scenario_ref1_optimal_policies_new.csv')

## Voila - Our optimal Policies! 

In [11]:
results_load.iloc[:,31:42]

,A.1_Expected Number of Deaths,A.1_Expected Annual Damage,A.1_Dike Investment Costs,A.2_Expected Number of Deaths,A.2_Expected Annual Damage,A.2_Dike Investment Costs,A.3_Expected Number of Deaths,A.3_Expected Annual Damage,A.3_Dike Investment Costs,RfR Total Costs,Expected Evacuation Costs
0,0.0,0.0,0.000000e+00,0.000000,0.000000e+00,0.000000e+00,0.000000,0.000000e+00,4.129222e+07,432800000.0,0.000000
1,0.0,0.0,4.229151e+07,0.009067,1.077004e+07,5.335857e+07,0.006258,3.932240e+06,2.029929e+07,0.0,0.000000
2,0.0,0.0,1.031623e+08,0.000000,0.000000e+00,0.000000e+00,0.000000,0.000000e+00,0.000000e+00,92100000.0,0.000000
3,0.0,0.0,6.650701e+07,0.000000,0.000000e+00,0.000000e+00,0.000000,0.000000e+00,2.029929e+07,92100000.0,0.000000
4,0.0,0.0,0.000000e+00,0.000000,0.000000e+00,4.784270e+07,0.018775,1.179672e+07,2.029929e+07,0.0,0.000000
...,...,...,...,...,...,...,...,...,...,...,...
75,0.0,0.0,0.000000e+00,0.009067,1.077004e+07,7.331545e+07,0.006258,3.932240e+06,2.029929e+07,0.0,0.000000
76,0.0,0.0,0.000000e+00,0.001627,1.284698e+07,3.824512e+07,0.000000,0.000000e+00,3.422156e+07,0.0,476.477660
77,0.0,0.0,0.000000e+00,0.004534,5.385022e+06,4.281842e+07,0.065988,4.202628e+07,0.000000e+00,0.0,0.000000
78,0.0,0.0,3.269490e+07,0.001627,1.284698e+07,3.824512e+07,0.009860,4.193614e+07,0.000000e+00,0.0,1738.450538


In [17]:
results_load.iloc[:,31:42].columns

Index(['A.1_Expected Number of Deaths', 'A.1_Expected Annual Damage',
       'A.1_Dike Investment Costs', 'A.2_Expected Number of Deaths',
       'A.2_Expected Annual Damage', 'A.2_Dike Investment Costs',
       'A.3_Expected Number of Deaths', 'A.3_Expected Annual Damage',
       'A.3_Dike Investment Costs', 'RfR Total Costs',
       'Expected Evacuation Costs'],
      dtype='object')

In [75]:
# from ema_workbench.analysis import parcoords

# paraxes = parcoords.ParallelAxes(parcoords.get_limits(results_load.iloc[:,31:34]), fontsize=9, rot=0)
# paraxes.plot(results_load.iloc[:,31:34], color=sns.color_palette()[0])

# paraxes1 = parcoords.ParallelAxes(parcoords.get_limits(results_load.iloc[:,34:37]), fontsize=9, rot=0)
# paraxes1.plot(results_load.iloc[:,34:37], color=sns.color_palette()[1])

# paraxes1 = parcoords.ParallelAxes(parcoords.get_limits(results_load.iloc[:,37:40]), fontsize=9, rot=0)
# paraxes1.plot(results_load.iloc[:,37:40], color=sns.color_palette()[2])

# paraxes1 = parcoords.ParallelAxes(parcoords.get_limits(results_load.iloc[:,40:43]), fontsize=9, rot=0)
# paraxes1.plot(results_load.iloc[:,40:43], color=sns.color_palette()[3])

# plt.show()

In [39]:
results_index = results_load.reset_index()

In [40]:
results_index

,index,0_RfR 0,0_RfR 1,0_RfR 2,1_RfR 0,1_RfR 1,1_RfR 2,2_RfR 0,2_RfR 1,2_RfR 2,...,A.1_Expected Annual Damage,A.1_Dike Investment Costs,A.2_Expected Number of Deaths,A.2_Expected Annual Damage,A.2_Dike Investment Costs,A.3_Expected Number of Deaths,A.3_Expected Annual Damage,A.3_Dike Investment Costs,RfR Total Costs,Expected Evacuation Costs
0,0,0,1,0,0,0,0,1,1,1,...,0.0,0.000000e+00,0.000000,0.000000e+00,0.000000e+00,0.000000,0.000000e+00,4.129222e+07,432800000.0,0.000000
1,1,0,0,0,0,0,0,0,0,0,...,0.0,4.229151e+07,0.009067,1.077004e+07,5.335857e+07,0.006258,3.932240e+06,2.029929e+07,0.0,0.000000
2,2,0,0,0,0,0,0,1,1,1,...,0.0,1.031623e+08,0.000000,0.000000e+00,0.000000e+00,0.000000,0.000000e+00,0.000000e+00,92100000.0,0.000000
3,3,0,0,0,0,0,0,1,1,1,...,0.0,6.650701e+07,0.000000,0.000000e+00,0.000000e+00,0.000000,0.000000e+00,2.029929e+07,92100000.0,0.000000
4,4,0,0,0,0,0,0,0,0,0,...,0.0,0.000000e+00,0.000000,0.000000e+00,4.784270e+07,0.018775,1.179672e+07,2.029929e+07,0.0,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
75,75,0,0,0,0,0,0,0,0,0,...,0.0,0.000000e+00,0.009067,1.077004e+07,7.331545e+07,0.006258,3.932240e+06,2.029929e+07,0.0,0.000000
76,76,0,0,0,0,0,0,0,0,0,...,0.0,0.000000e+00,0.001627,1.284698e+07,3.824512e+07,0.000000,0.000000e+00,3.422156e+07,0.0,476.477660
77,77,0,0,0,0,0,0,0,0,0,...,0.0,0.000000e+00,0.004534,5.385022e+06,4.281842e+07,0.065988,4.202628e+07,0.000000e+00,0.0,0.000000
78,78,0,0,0,0,0,0,0,0,0,...,0.0,3.269490e+07,0.001627,1.284698e+07,3.824512e+07,0.009860,4.193614e+07,0.000000e+00,0.0,1738.450538


### Plotting tradeoffs between outcomes using a Parallel coordinate plot. 

In [58]:
import plotly.express as px
# fig, ax = plt.subplots(figsize=(20,20))

fig = px.parallel_coordinates(results_index.iloc[:,32:43],color=results_index.index,
                              dimensions=results_load.iloc[:,31:42].columns
                              , width=2000, height=600)
fig.show()

In [67]:
give_me_deaths = results_index.loc[:,['index','A.1_Expected Number of Deaths','A.2_Expected Number of Deaths','A.3_Expected Number of Deaths']]

In [74]:
import plotly.express as px
# fig, ax = plt.subplots(figsize=(20,20))

fig = px.parallel_coordinates(give_me_deaths,color=give_me_deaths.index,
                              dimensions=give_me_deaths.iloc[:,1:4].columns
                              , width=900, height=600)
fig.show()

## A1 has 0 expected deaths but A2 and A3 do not. We need to make them 0 too for meeting our goal of safety of citizens. This is will be used in further analysis as a limiting criteria to have better policies. 

## The 80 optimal policies resulting from this analysis is used further in MORDM for getting robust polcies. 